In [24]:
import re
import gurobipy as gb
import numpy as np
import pandas as pd

import networkx as nx
import matplotlib.pyplot as plt
from gtfs_functions import Feed
from keplergl import KeplerGl
import geopandas as gpd
from shapely.geometry import Point, LineString

## Loading data


In [2]:
stops = pd.read_csv("./STM GTFS/stops.txt", sep=",")
stops = gpd.GeoDataFrame(
    stops, geometry=gpd.points_from_xy(stops.stop_lon, stops.stop_lat)
)
stops["stop_code"] = stops["stop_code"].astype(int)

stops

,stop_id,stop_code,stop_name,stop_lat,stop_lon,stop_url,location_type,parent_station,wheelchair_boarding,geometry
0,STATION_M118,10118,STATION ANGRIGNON,45.446466,-73.603118,NaN,1,NaN,1,POINT (-73.60312 45.44647)
1,43,10118,Station Angrignon,45.446466,-73.603118,http://www.stm.info/fr/infos/reseaux/metro/ang...,0,STATION_M118,1,POINT (-73.60312 45.44647)
2,43-01,10118,Station Angrignon,45.446319,-73.603835,NaN,2,STATION_M118,1,POINT (-73.60384 45.44632)
3,STATION_M120,10120,STATION MONK,45.451158,-73.593242,NaN,1,NaN,2,POINT (-73.59324 45.45116)
4,42,10120,Station Monk,45.451158,-73.593242,http://www.stm.info/fr/infos/reseaux/metro/monk,0,STATION_M120,2,POINT (-73.59324 45.45116)
...,...,...,...,...,...,...,...,...,...,...
9039,60987,60987,Institut national de santé publique (Sainte-Marie,45.433289,-73.907005,https://www.stm.info/fr/recherche#stq=60987,0,NaN,2,POINT (-73.90700 45.43329)
9040,61121,61121,du Souvenir / du Boulevard (Hôpital Ste-Anne),45.411347,-73.949240,https://www.stm.info/fr/recherche#stq=61121,0,NaN,2,POINT (-73.94924 45.41135)
9041,61253,61253,Lakeshore / Macdonald,45.403753,-73.940191,https://www.stm.info/fr/recherche#stq=61253,0,NaN,1,POINT (-73.94019 45.40375)
9042,61274,61274,SRB Pie-IX / Saint-Martin (zone B),45.610343,-73.662089,https://www.stm.info/fr/recherche#stq=61274,0,NaN,1,POINT (-73.66209 45.61034)


In [3]:
NUM_ROUTES = 5
distance_matrix = pd.read_json("distance_matrix.json")

In [4]:
# add depot to distance matrix
for i in range(NUM_ROUTES):
    distance_matrix.loc[i+1] = 0.0
    distance_matrix[i+1] = 0.0

distance_matrix

,51989,52053,52123,62063,52246,52321,52408,52447,52548,61743,...,53718,53719,53513,61728,62009,1,2,3,4,5
51989,0.000000,0.263512,0.427379,0.672261,0.815025,1.072613,1.388016,1.477391,1.854303,2.156428,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
52053,0.263512,0.000000,0.163867,0.408749,0.551513,0.809101,1.124504,1.213879,1.590791,1.892915,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
52123,0.427379,0.163867,0.000000,0.244882,0.387646,0.645234,0.960637,1.050012,1.426924,1.729048,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
62063,0.672261,0.408749,0.244882,0.000000,0.142764,0.400352,0.715755,0.805130,1.182042,1.484166,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
52246,0.815025,0.551513,0.387646,0.142764,0.000000,0.257588,0.572991,0.662366,1.039278,1.341402,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
np.nanmax(distance_matrix.to_numpy())

28.9222631044

In [6]:
distance_matrix.fillna(29, inplace=True)

In [7]:
distance_matrix

,51989,52053,52123,62063,52246,52321,52408,52447,52548,61743,...,53718,53719,53513,61728,62009,1,2,3,4,5
51989,0.000000,0.263512,0.427379,0.672261,0.815025,1.072613,1.388016,1.477391,1.854303,2.156428,...,29.0,29.0,29.0,29.0,29.0,0.0,0.0,0.0,0.0,0.0
52053,0.263512,0.000000,0.163867,0.408749,0.551513,0.809101,1.124504,1.213879,1.590791,1.892915,...,29.0,29.0,29.0,29.0,29.0,0.0,0.0,0.0,0.0,0.0
52123,0.427379,0.163867,0.000000,0.244882,0.387646,0.645234,0.960637,1.050012,1.426924,1.729048,...,29.0,29.0,29.0,29.0,29.0,0.0,0.0,0.0,0.0,0.0
62063,0.672261,0.408749,0.244882,0.000000,0.142764,0.400352,0.715755,0.805130,1.182042,1.484166,...,29.0,29.0,29.0,29.0,29.0,0.0,0.0,0.0,0.0,0.0
52246,0.815025,0.551513,0.387646,0.142764,0.000000,0.257588,0.572991,0.662366,1.039278,1.341402,...,29.0,29.0,29.0,29.0,29.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
nodes = list(distance_matrix.index)
num_nodes = len(nodes)

# Vehicle Routing - Bus Route Optimization

In [9]:
vrp = gb.Model("Montreal Bus Routing")
vrp.Params.MIPGap = 0.3
vrp.Params.TimeLimit = 30
start_node_index = 173


# Decision variables
x = vrp.addVars(nodes, nodes, vtype=gb.GRB.BINARY, name="x")

u = vrp.addVars(nodes, vtype=gb.GRB.INTEGER, name="u", ub=num_nodes)

# Objective function
vrp.setObjective(
    gb.quicksum(distance_matrix.loc[i, j] * x[i, j] for i in nodes for j in nodes),
    gb.GRB.MINIMIZE,
)

# Constraints
vrp.addConstrs(
    (gb.quicksum(x[i, j] for j in nodes if j != i) == 1 for i in nodes),
    name="outgoing",
)

vrp.addConstrs(
    (gb.quicksum(x[j, i] for j in nodes if j != i) == 1 for i in nodes),
    name="ingoing",
)

vrp.addConstrs(
    (
        u[i] - u[j] + num_nodes * x[i, j] <= num_nodes - 1
        for i in nodes
        for j in nodes
        if i != j and i != nodes[start_node_index] and j != nodes[start_node_index]
    ),
    name="subtour_elimination",
)

vrp.addConstrs(
    (u[i] >= 2 for i in nodes if i != nodes[start_node_index]), name="lower bound for u"
)

vrp.addConstr(
    u[nodes[start_node_index]] == 1,
    name="start node index is 1",
)

vrp.optimize()

Set parameter Username


INFO:gurobipy.gurobipy:Set parameter Username


Academic license - for non-commercial use only - expires 2024-08-30


INFO:gurobipy.gurobipy:Academic license - for non-commercial use only - expires 2024-08-30


Set parameter MIPGap to value 0.3


...

INFO:gurobipy.gurobipy:


Optimal solution found (tolerance 3.00e-01)


INFO:gurobipy.gurobipy:Optimal solution found (tolerance 3.00e-01)


Best objective 5.267839907950e+01, best bound 4.779634992895e+01, gap 9.2676%


INFO:gurobipy.gurobipy:Best objective 5.267839907950e+01, best bound 4.779634992895e+01, gap 9.2676%


In [10]:
nodes[start_node_index]

1

In [11]:
print("Minimum distance: ", vrp.objVal)
print("Optimal path: ")
tour = []
for v in vrp.getVars():
    if v.varName.startswith("u"):
        tour.append(v)

for s in sorted(tour, key=lambda x: x.x):
    print(s.varName, s.x)

Minimum distance:  52.6783990795
Optimal path: 
u[1] 1.0
u[52517] 2.0
u[52439] 3.0
u[52412] 4.0
u[52499] 5.0
u[52588] 6.0
u[52666] 7.0
u[52705] 8.0
u[52826] 9.0
u[60426] 10.0
u[52992] 11.0
u[52975] 12.0
u[61765] 13.0
u[52945] 14.0
u[52913] 15.0
u[61744] 16.0
u[58803] 17.0
u[61743] 18.0
u[52408] 19.0
u[52321] 20.0
u[52246] 21.0
u[62063] 22.0
u[52053] 23.0
u[51989] 24.0
u[52123] 25.0
u[52447] 26.0
u[52548] 27.0
u[59235] 28.0
u[59223] 29.0
u[61745] 30.0
u[52926] 31.0
u[53898] 32.0
u[53899] 33.0
u[52912] 34.0
u[52893] 35.0
u[52887] 36.0
u[52871] 37.0
u[53900] 38.0
u[60540] 39.0
u[56719] 40.0
u[56718] 41.0
u[56715] 42.0
u[56638] 43.0
u[56636] 44.0
u[56631] 45.0
u[56629] 46.0
u[56627] 47.0
u[56621] 48.0
u[56619] 49.0
u[56706] 50.0
u[3] 51.0
u[54138] 52.0
u[54281] 53.0
u[51220] 54.0
u[54186] 55.0
u[53697] 56.0
u[55311] 57.0
u[55317] 58.0
u[55218] 59.0
u[60386] 60.0
u[55315] 61.0
u[55272] 62.0
u[55285] 63.0
u[55296] 64.0
u[55306] 65.0
u[55302] 66.0
u[60507] 67.0
u[60387] 68.0
u[61529] 69.0
u[5

In [12]:
# Plotting the directed graph
G = nx.DiGraph()

for i in nodes:
    for j in nodes:
        if x[i, j].x == 1:
            G.add_edge(
                i, j, weight=distance_matrix[i][j], step=u[i].x, label=f"{i}-{j}"
            )

pos = nx.spring_layout(G)
steps = nx.get_edge_attributes(G, "step")

from pyvis.network import Network

net = Network(
    directed=True,
    notebook=True,
    select_menu=True,
    filter_menu=True,
    cdn_resources="remote",
    neighborhood_highlight=True,
)

net.from_nx(G)
net.show_buttons()


net.show("example.html")

example.html


In [20]:
# convert tour into dictionary of steps and nodes

tour_dict = {}

for node in tour:
    node_name = node.varName

    node_name = re.sub(r"[u\[\]]", "", node_name)
    tour_dict[node.x] = int(node_name)

tour_df = (
    pd.DataFrame.from_dict(tour_dict, orient="index", columns=["node"])
    .sort_index()
    .reset_index()
    .rename(columns={"index": "step"})
)

tour_df = tour_df.merge(stops, left_on="node", right_on="stop_code", how="left")

tour_df['stop_lat'] = tour_df['stop_lat'].fillna(45.508888)
tour_df['stop_lon'] = tour_df['stop_lon'].fillna(-73.561668)

tour_df

,step,node,stop_id,stop_code,stop_name,stop_lat,stop_lon,stop_url,location_type,parent_station,wheelchair_boarding,geometry
0,1.0,1,NaN,NaN,NaN,45.508888,-73.561668,NaN,NaN,NaN,NaN,None
1,2.0,52517,52517,52517.0,Station Place-des-Arts (Président-Kennedy / De...,45.507283,-73.569604,https://www.stm.info/fr/recherche#stq=52517,0.0,NaN,1.0,POINT (-73.56960 45.50728)
2,3.0,52439,52439,52439.0,du Président-Kennedy / Aylmer,45.505745,-73.571114,https://www.stm.info/fr/recherche#stq=52439,0.0,NaN,1.0,POINT (-73.57111 45.50574)
3,4.0,52412,52412,52412.0,Station McGill (Robert-Bourassa / Président-Ke...,45.504430,-73.572240,https://www.stm.info/fr/recherche#stq=52412,0.0,NaN,1.0,POINT (-73.57224 45.50443)
4,5.0,52499,52499,52499.0,Robert-Bourassa / Sainte-Catherine,45.503361,-73.570056,https://www.stm.info/fr/recherche#stq=52499,0.0,NaN,1.0,POINT (-73.57006 45.50336)
...,...,...,...,...,...,...,...,...,...,...,...,...
173,174.0,53457,53457,53457.0,Sherbrooke / du Tricentenaire,45.652566,-73.510681,https://www.stm.info/fr/recherche#stq=53457,0.0,NaN,1.0,POINT (-73.51068 45.65257)
174,175.0,54248,54248,54248.0,Sherbrooke / No 12572,45.650351,-73.511446,https://www.stm.info/fr/recherche#stq=54248,0.0,NaN,1.0,POINT (-73.51145 45.65035)
175,176.0,53435,53435,53435.0,Sherbrooke / 16e Avenue,45.646831,-73.512662,https://www.stm.info/fr/recherche#stq=53435,0.0,NaN,1.0,POINT (-73.51266 45.64683)
176,177.0,61731,61731,61731.0,Sherbrooke / Saint-Jean-Baptiste,45.644657,-73.513411,https://www.stm.info/fr/recherche#stq=61731,0.0,NaN,1.0,POINT (-73.51341 45.64466)


In [22]:
map = KeplerGl(height=700)

map.add_data(
    data=tour_df[["stop_lat", "stop_lon", "stop_name", "step"]],
    name="stops_in_trip",
)

map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'stops_in_trip': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 1…